**Importing Necessary Packages**

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

**Mounting Google Drive**

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Using Pre-trained yolov3 weights and cfg file.**

Two Pretrained Files are being Used:

1) yolov3.weights

2) yolov3.cfg

In [3]:
net = cv2.dnn.readNet('gdrive/My Drive/Colab Notebooks/yolov3.weights','gdrive/My Drive/Colab Notebooks/yolov3.cfg')

**Using coco.names for getting names of 80 different classes.**

In [4]:
classes = []
with open('gdrive/My Drive/Colab Notebooks/coco.names', 'r') as f:
  classes = f.read().splitlines()
print(len(classes))

80


**Installing and importing mediapy for video capture and process**

In [5]:
!pip install mediapy

In [6]:
import mediapy as media

**Print shape, number of frames and fps of the video**

In [7]:
VIDEO = 'gdrive/My Drive/Colab Notebooks/DL_OBJ_Detection.mp4'
with media.VideoReader(VIDEO) as reader:
  print(f'num_images={reader.num_images} shape={reader.shape} fps={reader.fps}')

num_images=4845 shape=(720, 1280) fps=30.0


**Reading Videos from the Google drive, Processing Frame by Frame and writing them back to the drive.**

In [ ]:
new_file = 'gdrive/My Drive/Colab Notebooks/out1.mp4'
with media.VideoWriter(new_file,shape=(reader.shape)) as writer:
  cap = cv2.VideoCapture(VIDEO)
  count = 0
  while True:
    _, img = cap.read()
    if count>=reader.num_images-5:
      break
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0),swapRB=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
      for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
          center_x = int(detection[0]*width)
          center_y = int(detection[1]*height)
          w = int(detection[2]*width)
          h = int(detection[3]*height)

          x = int(center_x - w/2)
          y = int(center_y - h/2)

          boxes.append([x,y,w,h])
          confidences.append(float(confidence))
          class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0,255,size=(len(boxes),3))
    if len(boxes)>0:
      for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i],2))
        color = colors[i]
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        cv2.putText(img, label + " " + confidence, (x,y+20), font, 2, (255,255,255), 2)

    #plt.imshow(img)
    #plt.show()
    writer.add_image(img)
    #cv2.imwrite('gdrive/My Drive/Colab Notebooks/image_frames/'+str(count)+'.png',img)
    count += 1
    if count%100==0:
      print(count)

  cap.release()
media.show_video(media.read_video(new_file), height=90)